In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import wandb
import math
sys.path.append('../..')

from LightningModules.GNNNodeEmbedding.Models.gnn_embedding import InteractionGNN

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
def kaiming_init(model):
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            param.data.fill_(0)
        elif name.startswith("layers.0"):  # The first layer does not have ReLU applied on its input
            param.data.normal_(0, 1 / math.sqrt(param.shape[1]))
        else:
            param.data.normal_(0, math.sqrt(2) / math.sqrt(param.shape[1]))

In [5]:
def load_from_pretrained(model, path):
    
    checkpoint = torch.load(path)
    state_dict = checkpoint["state_dict"]
    names = [i for i in state_dict]
    for i in names:
        if "output_layer" in i:
            del state_dict[i]
    model.load_state_dict(state_dict, strict=False)
    del state_dict
    
    return model

## Construct PyLightning model

In [ ]:
with open("gnn_node_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
model = InteractionGNN(hparams)
# model = load_from_pretrained(model, "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_embedding/ITk_gnn_embedding/37903mw4/checkpoints/last.ckpt")

## Metric Learning

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
# kaiming_init(model)
logger = WandbLogger(project="ITk_gnn_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_embedding/")
trainer.fit(model)

## Sweep

In [6]:
run_name = input()

 eff&pur


In [7]:
with open("gnn_node_embedding_sweep.yaml") as f:
        sweep_hparams = yaml.load(f, Loader=yaml.FullLoader)
with open("gnn_node_embedding_default.yaml") as f:
        default_hparams = yaml.load(f, Loader=yaml.FullLoader)

In [8]:
sweep_configuration = {
    "name": run_name,
    "project": "ITk_barrell_gnn_embedding",
    "metric": {"name": "pur", "goal": "maximize"},
    "method": "grid",
    "parameters": sweep_hparams
}

In [9]:
def training():
    wandb.init()
    model = InteractionGNN({**default_hparams, **wandb.config})

    # kaiming_init(model)
    
    checkpoint_callback = ModelCheckpoint(
        monitor='pur',
        mode="max",
        save_top_k=2,
        save_last=True)

    logger = WandbLogger()
    trainer = Trainer(gpus=1, max_epochs=default_hparams["max_epochs"], log_every_n_steps = 50, logger=logger, callbacks=[checkpoint_callback], default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_embedding/")
    trainer.fit(model)

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project = "ITk_barrel_gnn_embedding")

# run the sweep
wandb.agent(sweep_id, function=training)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 69wgxiuj
Sweep URL: https://wandb.ai/exatrkx/ITk_barrel_gnn_embedding/sweeps/69wgxiuj


wandb: Agent Starting Run: 7sqon2zn with config:
wandb: 	toy_eff: 1
wandb: 	toy_pur: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/global/homes/r/ryanliu/.conda/envs/pyg2/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'toy_eff' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'toy_pur' was locked by 'sweep' (ignored update).

  | Name         | Type             | Params
--------------------------------------------------
0 | cos          | CosineSimilarity | 0     
1 | node_encoder | Sequential       | 34.2 K
2 | edge_encoder | Sequential       | 34.4 K
3 | gnn_blocks   | ModuleList       | 1.2 M 
4 | output_layer | Sequential       | 69.1 K
--------------------------------------------

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 20: 100%|█████████▉| 1000/1005 [01:46<00:00,  9.38it/s, loss=0.0403, v_num=1t9c]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/5 [00:00<?, ?it/s]

/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = si


Epoch 21:   0%|          | 0/1005 [00:00<?, ?it/s, loss=0.0403, v_num=1t9c]           

/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNNodeEmbedding/utils.py:266: RuntimeWarning: invalid value encountered in sqrt
  y = si

Epoch 21:  55%|█████▌    | 555/1005 [00:55<00:44, 10.03it/s, loss=0.0404, v_num=1t9c]